In [1]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import scipy
from scipy.stats import norm
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

from sklearn.metrics import accuracy_score

C:\Users\tulasi ram\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#importing sqlite3
import sqlite3
#creating connection with the DataBase file.
#Here in final.sqlite  already preprocessed data is added in one column named Cleaned Text
con=sqlite3.connect('C:\\Users\\tulasi ram\\Desktop\\amazon_food_reviews\\final.sqlite')
final_data=pd.read_sql_query("""SELECT * FROM Reviews """, con)
#it is always required to close the connection
con.close()

In [3]:
print(final_data.columns)
print(final_data.shape)
final_data['Score'].value_counts()

Index(['index', 'Id', 'ProductId', 'UserId', 'ProfileName',
       'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time',
       'Summary', 'Text', 'CleanedText', 'WordCount'],
      dtype='object')
(364171, 13)


positive    307061
negative     57110
Name: Score, dtype: int64

In [4]:
#Selecting the sample of the dataframe for the operations


Negative = final_data.loc[final_data['Score'] == 'negative']
Positive = final_data.loc[final_data['Score'] == 'positive']

print(Negative.shape)
print(Positive.shape)

#As the data looks to be imbalanced we are sampling the data and making it balanced
np.random.seed(100)
indices=np.random.randint(0, 57110, size=5000)
Negative = Negative.iloc[indices]
print('Negative reviews shape', Negative.shape)
indices=np.random.randint(0, 307061, size=5000)
Positive = Positive.iloc[indices]
print('Positive reviews shape', Positive.shape)
sampled_data=pd.concat([Negative,Positive],ignore_index=False)
print('Sampled data shape', sampled_data.shape)

(57110, 13)
(307061, 13)
Negative reviews shape (5000, 13)
Positive reviews shape (5000, 13)
Sampled data shape (10000, 13)


In [5]:
sorted_data=sampled_data.sort_values(by=['Time'])

sorted_data.head()


,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText,WordCount
308,346041,374343,B00004CI84,A1B2IZU1JLZA6,Wes,19,23,negative,948240000,WARNING: CLAMSHELL EDITION IS EDITED TV VERSION,"I, myself always enjoyed this movie, it's very...",b'alway enjoy movi funni entertain didnt hesit...,32
362,346077,374382,B00004CI84,A3C3BAQDZWH5YE,Kushana no shinryaku (Kushana's invasion),0,1,positive,1014681600,...,"It was on the other night, and, having been a ...",b'night big fan cartoon shown decid watch also...,38
365,346040,374342,B00004CI84,A10L8O1ZMUIMR2,G. Kleinschmidt,61,79,negative,1040947200,Great movie turned bad,"Just to let you know, this movie is one of my ...",b'let know movi one person favorit ghost movi ...,114
2293,38888,42226,B0000A0BS8,A23GFTVIETX7DS,Debbie Lee Wesselmann,5,5,positive,1067904000,Five stars if you like Starbucks,This blend is one of Starbucks' gentler blends...,b'blend one starbuck gentler blend like tast s...,59
4682,224637,243579,B0000DIYKD,AYHW6HJSUCSAE,"""insolent_shoeshine_grrl""",11,13,positive,1072656000,Fantastic Real Licorice From The Land Down Under,I recently purchased a bag of Kookaburra Black...,b'recent purchas bag kookaburra black licoric ...,95


**Analysis using bag of words vectors**

In [6]:
#Bow
bow_vectors=CountVectorizer(binary = True).fit_transform(sorted_data['CleanedText'].values)
print(bow_vectors.shape)
np.save('bow_vectors1',bow_vectors.toarray())
final_bow=bow_vectors.toarray()
labels=sorted_data['Score']

(10000, 13237)


In [7]:
from sklearn.preprocessing import StandardScaler
standardized_data=StandardScaler().fit_transform(final_bow)
print(standardized_data.shape)

C:\Users\tulasi ram\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(10000, 13237)


In [8]:
from sklearn import cross_validation
#Splitting the data into train and test 
X_1, X_test, Y_1, Y_test=cross_validation.train_test_split(standardized_data , labels, test_size=0.35 ,random_state=0)


In [9]:


#Using GridSearchCV
#As it is balanced data taking scoring parameter as accuracy is also good
tuned_parameters = [{'C': [10**-4,10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3 ,10**4]}]
grid_search = GridSearchCV(LogisticRegression(), tuned_parameters, scoring = 'accuracy', cv=10)
grid_search.fit(X_1,Y_1)

print(grid_search.best_estimator_)
print("The accuracy of the model is %f%% ",grid_search.score(X_test, Y_test)*100)


LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
The accuracy of the model is %f%%  84.17142857142858


In [10]:
#Using RandomSearch

parameters = {'C' : list(filter(lambda x: x>0, norm.rvs(size=50)))}
random_search = RandomizedSearchCV(LogisticRegression(), parameters, scoring = 'accuracy', cv=5)
random_search.fit(X_1,Y_1)

print(random_search.best_estimator_)
print("\nThe accuracy of the model is %f%% ",random_search.score(X_test, Y_test)*100)


LogisticRegression(C=0.10535917233957426, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

The accuracy of the model is %f%%  81.77142857142857


**The accuracy using randomized search is almost same as accuracy of grid search**

In [12]:


clf = LogisticRegression(C=0.001, penalty='l2');
clf.fit(X_1, Y_1);
w = clf.coef_

print(w[:,0:9])
print(np.count_nonzero(w))
# predict the response
pred = clf.predict(X_test)

# evaluate accuracy
acc = accuracy_score(Y_test, pred) * 100
print('\nThe accuracy of the LR classifier is %f%%' , acc)

[[-0.00285247 -0.00629317  0.00174559 -0.0029502   0.00985739  0.00119812
   0.00526796  0.00753386  0.00222016]]
13237

The accuracy of the LR classifier is %f%% 84.17142857142858


In [13]:
List=[1,0.1,0.01,0.001,0.0001]
for i in List:
    clf = LogisticRegression(C=i, penalty='l1');
    clf.fit(X_1, Y_1);
    w = clf.coef_
    
    print('\nThe non zero values in the weight vector with C value', i ,'is',np.count_nonzero(w))
    # predict the response
    pred = clf.predict(X_test)
    # evaluate accuracy
    acc = accuracy_score(Y_test, pred) * 100
    print('The accuracy of the LR classifier with C value',i ,' is %f%%' , acc)


The non zero values in the weight vector with C value 1 is 2832
The accuracy of the LR classifier with C value 1  is %f%% 82.17142857142858

The non zero values in the weight vector with C value 0.1 is 2402
The accuracy of the LR classifier with C value 0.1  is %f%% 83.2

The non zero values in the weight vector with C value 0.01 is 196
The accuracy of the LR classifier with C value 0.01  is %f%% 81.05714285714286

The non zero values in the weight vector with C value 0.001 is 0
The accuracy of the LR classifier with C value 0.001  is %f%% 49.94285714285714

The non zero values in the weight vector with C value 0.0001 is 0
The accuracy of the LR classifier with C value 0.0001  is %f%% 49.94285714285714


**Conclusion**

It is clear that decreasing the values of C i.e, increaing the values of lambda sparsity of the weight vector is increasing and when C=0.001 the weight vector becomes completely 0.

It is also understood that while increasing the values of lambda accuracy is decreasing and error is increasing finally performance of the model is decreasing

Here when C values are less than 0.001 the accuracy is even less than 50 percent. 

**Analysis using tfidf vectors**

In [15]:
#TF-IDF
tf_idf_vect = TfidfVectorizer(ngram_range=(1,1))
final_tf_idf = tf_idf_vect.fit_transform(sorted_data['CleanedText'].values)
print(final_tf_idf.shape)
final_tf_idf_vectors=final_tf_idf.toarray()

(10000, 13237)


In [50]:
#Standardizing the data .
from sklearn.preprocessing import StandardScaler
standardized_data=StandardScaler().fit_transform(final_tf_idf_vectors)
print(standardized_data.shape)

(10000, 13237)


In [52]:
#Splitting the data into train and test 
X_1, X_test, Y_1, Y_test=cross_validation.train_test_split(standardized_data , labels, test_size=0.35 ,random_state=0)


In [53]:
#Using GridSearchCV
#As it is balanced data taking scoring parameter as accuracy is also good
tuned_parameters = [{'C': [10**-4,10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3 ,10**4]}]
grid_search = GridSearchCV(LogisticRegression(), tuned_parameters, scoring = 'accuracy', cv=10)
grid_search.fit(X_1,Y_1)

print(grid_search.best_estimator_)
print("The accuracy of the model is %f%% ",grid_search.score(X_test, Y_test)*100)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
The accuracy of the model is %f%%  83.71428571428572


In [54]:
#Using RandomSearch

parameters = {'C' : list(filter(lambda x: x>0, norm.rvs(size=50)))}
random_search = RandomizedSearchCV(LogisticRegression(), parameters, scoring = 'accuracy', cv=5)
random_search.fit(X_1,Y_1)

print(random_search.best_estimator_)
print("\nThe accuracy of the model is %f%% ",random_search.score(X_test, Y_test)*100)


LogisticRegression(C=0.3280456967491141, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

The accuracy of the model is %f%%  80.11428571428571


**Here also the accuracy using randomized search is almost same as accuracy of grid search**

In [56]:


clf = LogisticRegression(C=0.0001, penalty='l2');
clf.fit(X_1, Y_1);
w = clf.coef_

print(w[:,0:9])
print(np.count_nonzero(w))
# predict the response
pred = clf.predict(X_test)

# evaluate accuracy
acc = accuracy_score(Y_test, pred) * 100
print('\nThe accuracy of the LR classifier is %f%%' , acc)

[[-0.00260612 -0.00299843  0.00160532 -0.00103442  0.00300244  0.00104636
   0.00257714  0.00319687  0.00102679]]
13237

The accuracy of the LR classifier is %f%% 83.71428571428572


In [57]:
List=[1,0.1,0.01,0.001,0.0001]
for i in List:
    clf = LogisticRegression(C=i, penalty='l1');
    clf.fit(X_1, Y_1);
    w = clf.coef_
    
    print('\nThe non zero values in the weight vector with C value', i ,'is',np.count_nonzero(w))
    # predict the response
    pred = clf.predict(X_test)
    # evaluate accuracy
    acc = accuracy_score(Y_test, pred) * 100
    print('The accuracy of the LR classifier with C value',i ,' is %f%%' , acc)


The non zero values in the weight vector with C value 1 is 3427
The accuracy of the LR classifier with C value 1  is %f%% 82.31428571428572

The non zero values in the weight vector with C value 0.1 is 2915
The accuracy of the LR classifier with C value 0.1  is %f%% 83.54285714285714

The non zero values in the weight vector with C value 0.01 is 186
The accuracy of the LR classifier with C value 0.01  is %f%% 80.91428571428571

The non zero values in the weight vector with C value 0.001 is 0
The accuracy of the LR classifier with C value 0.001  is %f%% 49.94285714285714

The non zero values in the weight vector with C value 0.0001 is 0
The accuracy of the LR classifier with C value 0.0001  is %f%% 49.94285714285714


**Conclusion**

It is clear that decreasing the values of C i.e, increaing the values of lambda sparsity of the weight vector is increasing and when C=0.001 the weight vector becomes completely 0.

It is also understood that while increasing the values of lambda accuracy is decreasing and error is increasing finally performance of the model is decreasing

Here when C values are less than 0.001 the accuracy is even less than 50 percent. 

**Analysis using avg W2V vectors**

In [25]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned


In [27]:

# Train our own Word2Vec model using our own text corpus
# cleaning the word of any html-tags
#cleaning the word of any punctuation or special characters
import gensim
i=0
list_of_sent=[]
for sent in sorted_data['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)
    

In [41]:
#Word2Vec
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)    


In [42]:
w2v_model.save('word2vec')

In [43]:
#AvgW2V Model
sent_vectors = []; 
for sent in list_of_sent: 
    sent_vec = np.zeros(50) 
    cnt_words =0; 
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)

In [44]:
#Standardizing the data .
from sklearn.preprocessing import StandardScaler
standardized_data=StandardScaler().fit_transform(sent_vectors)
print(standardized_data.shape)

(10000, 50)


In [45]:

#Splitting the data into train and test 
X_1, X_test, Y_1, Y_test=cross_validation.train_test_split(standardized_data , labels, test_size=0.35 ,random_state=0)


In [46]:


#Using GridSearchCV
#As it is balanced data taking scoring parameter as accuracy is also good
tuned_parameters = [{'C': [10**-4,10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3 ,10**4]}]
grid_search = GridSearchCV(LogisticRegression(), tuned_parameters, scoring = 'accuracy', cv=10)
grid_search.fit(X_1,Y_1)

print(grid_search.best_estimator_)
print("The accuracy of the model is %f%% ",grid_search.score(X_test, Y_test)*100)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
The accuracy of the model is %f%%  77.45714285714286


In [47]:
#Using RandomSearch

parameters = {'C' : list(filter(lambda x: x>0, norm.rvs(size=50)))}
random_search = RandomizedSearchCV(LogisticRegression(), parameters, scoring = 'accuracy', cv=5)
random_search.fit(X_1,Y_1)

print(random_search.best_estimator_)
print("\nThe accuracy of the model is %f%% ",random_search.score(X_test, Y_test)*100)


LogisticRegression(C=0.46819318791272424, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

The accuracy of the model is %f%%  77.57142857142857


In [48]:


clf = LogisticRegression(C=0.46, penalty='l2');
clf.fit(X_1, Y_1);
w = clf.coef_

print(w[:,0:9])
print(np.count_nonzero(w))
# predict the response
pred = clf.predict(X_test)

# evaluate accuracy
acc = accuracy_score(Y_test, pred) * 100
print('\nThe accuracy of the LR classifier is %f%%' , acc)

[[-0.00822811 -0.33204779  0.30584753 -0.58985789 -0.22889732 -0.00666807
   0.47102095 -0.27127216 -0.49164818]]
50

The accuracy of the LR classifier is %f%% 77.60000000000001


In [49]:
List=[1,0.1,0.01,0.001,0.0001]
for i in List:
    clf = LogisticRegression(C=i, penalty='l1');
    clf.fit(X_1, Y_1);
    w = clf.coef_
    
    print('\nThe non zero values in the weight vector with C value', i ,'is',np.count_nonzero(w))
    # predict the response
    pred = clf.predict(X_test)
    # evaluate accuracy
    acc = accuracy_score(Y_test, pred) * 100
    print('The accuracy of the LR classifier with C value',i ,' is %f%%' , acc)


The non zero values in the weight vector with C value 1 is 47
The accuracy of the LR classifier with C value 1  is %f%% 77.42857142857143

The non zero values in the weight vector with C value 0.1 is 40
The accuracy of the LR classifier with C value 0.1  is %f%% 76.97142857142858

The non zero values in the weight vector with C value 0.01 is 23
The accuracy of the LR classifier with C value 0.01  is %f%% 73.94285714285715

The non zero values in the weight vector with C value 0.001 is 0
The accuracy of the LR classifier with C value 0.001  is %f%% 49.94285714285714

The non zero values in the weight vector with C value 0.0001 is 0
The accuracy of the LR classifier with C value 0.0001  is %f%% 49.94285714285714


**Using Google W2V**

In [67]:
# Using Google News Word2Vectors
from gensim.models import Word2Vec
from gensim.models import KeyedVectors


In [69]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [72]:
#AvgW2V Model
sent_vectors_googlew2v = []; 
for sent in list_of_sent: 
    sent_vec = np.zeros(300) 
    cnt_words =0; 
    for word in sent:
        try:
            vec = model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors_googlew2v.append(sent_vec)

In [73]:
#Standardizing the data .
from sklearn.preprocessing import StandardScaler
standardized_data=StandardScaler().fit_transform(sent_vectors_googlew2v)
print(standardized_data.shape)

(10000, 300)


In [74]:

#Splitting the data into train and test 
X_1, X_test, Y_1, Y_test=cross_validation.train_test_split(standardized_data , labels, test_size=0.35 ,random_state=0)


In [75]:


#Using GridSearchCV
#As it is balanced data taking scoring parameter as accuracy is also good
tuned_parameters = [{'C': [10**-4,10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3 ,10**4]}]
grid_search = GridSearchCV(LogisticRegression(), tuned_parameters, scoring = 'accuracy', cv=10)
grid_search.fit(X_1,Y_1)

print(grid_search.best_estimator_)
print("The accuracy of the model is %f%% ",grid_search.score(X_test, Y_test)*100)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
The accuracy of the model is %f%%  83.57142857142857


In [76]:
#Using RandomSearch

parameters = {'C' : list(filter(lambda x: x>0, norm.rvs(size=50)))}
random_search = RandomizedSearchCV(LogisticRegression(), parameters, scoring = 'accuracy', cv=5)
random_search.fit(X_1,Y_1)

print(random_search.best_estimator_)
print("\nThe accuracy of the model is %f%% ",random_search.score(X_test, Y_test)*100)


LogisticRegression(C=0.10750374094965869, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

The accuracy of the model is %f%%  83.0


In [77]:
List=[1,0.1,0.01,0.001,0.0001]
for i in List:
    clf = LogisticRegression(C=i, penalty='l1');
    clf.fit(X_1, Y_1);
    w = clf.coef_
    
    print('\nThe non zero values in the weight vector with C value', i ,'is',np.count_nonzero(w))
    # predict the response
    pred = clf.predict(X_test)
    # evaluate accuracy
    acc = accuracy_score(Y_test, pred) * 100
    print('The accuracy of the LR classifier with C value',i ,' is %f%%' , acc)


The non zero values in the weight vector with C value 1 is 295
The accuracy of the LR classifier with C value 1  is %f%% 82.94285714285714

The non zero values in the weight vector with C value 0.1 is 231
The accuracy of the LR classifier with C value 0.1  is %f%% 83.28571428571429

The non zero values in the weight vector with C value 0.01 is 79
The accuracy of the LR classifier with C value 0.01  is %f%% 80.82857142857142

The non zero values in the weight vector with C value 0.001 is 0
The accuracy of the LR classifier with C value 0.001  is %f%% 49.94285714285714

The non zero values in the weight vector with C value 0.0001 is 0
The accuracy of the LR classifier with C value 0.0001  is %f%% 49.94285714285714


**FINAL CONCLUSION**

Of all these types of vectors BOW vectors is giving slightly more accuracy than other vectors